In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"/Users/anasnicholls/Desktop/CS238FinalProject/Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using Pkg; Pkg.add("DiscreteValueIteration")
using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `~/Desktop/CS238FinalProject/Project.toml`
 [no changes]
  Updating `~/Desktop/CS238FinalProject/Manifest.toml`
 [no changes]


In [3]:

sensor = Bumper()
config = 3 # 1,2, or 3
num_y_pts = 50
num_x_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
println(sspace)
aspace = vec(collect(RoombaAct(v, om) for v in [0,1,10], om in [-1,0,1]))
println(aspace)
@requirements_info SparseValueIterationSolver() RoombaMDP(config=config, sspace=sspace, aspace=aspace)
mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace)
println(mdp)
m = RoombaPOMDP(sensor=sensor, mdp=mdp)
println("hi")
solver = SparseValueIterationSolver(max_iterations=100, belres=1e-3) # initializes the Solver type
#solver = ValueIterationSolver(max_iterations=100, belres=1e-3) # initializes the Solver type
println("here")
policy = solve(solver, mdp) # runs value iterations
println("here")
#println(policy)
# for elem in sspace
#     a = action(policy, elem)
#     println(a)
# end


DiscreteRoombaStateSpace(1.0204081632653061, 1.0204081632653061, 0.3306939635357677, [-30.0, 20.0], [-30.0, 20.0])
RoombaAct[[0.0, -1.0], [1.0, -1.0], [10.0, -1.0], [0.0, 0.0], [1.0, 0.0], [10.0, 0.0], [0.0, 1.0], [1.0, 1.0], [10.0, 1.0]]
INFO: POMDPs.jl requirements for solve(::SparseValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::SparseValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::RoombaMDP)
  [✔] n_states(::RoombaMDP)
  [✔] n_actions(::RoombaMDP)
  [✔] transition(::RoombaMDP, ::RoombaState, ::RoombaAct)
  [✔] reward(::RoombaMDP, ::RoombaState, ::RoombaAct, ::RoombaState)
  [✔] stateindex(::RoombaMDP, ::RoombaState)
  [✔] actionindex(::RoombaMDP, ::RoombaAct)
  [✔] actions(::RoombaMDP, ::RoombaState)
  [✔] support(::Deterministic)
  [✔] pdf(::Deterministic, ::RoombaState)
For ordered_states(::Union{MDP,POMDP}) (in solve(::SparseValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] states(::RoombaMDP)
For 

In [4]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(mdp, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5
print("sampler")

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

sampler

## Create a Baseline Policy- Discretized Value Iteration

In [5]:
# Define the policy to test
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

# extract goal for heuristic controller
goal_xy = get_goal_xy(m)

# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    # compute mean belief of a subset of particles
    #println("policy")
    println("here")
#     if p.ts < 10
#         p.ts += 1
#         return RoombaAct(0.,1.0) # all actions are of type RoombaAct(v,om)
#     end
#     p.ts += 1
    
#     v = 2.5
    s = mean(b)
    return action(policy, s)
#     x,y,th = s[1:3]
    
#     ## If you hit a wall:
#     if wall_contact(m, s)
#         randomAngle = rand(r[1:3]) * pi*2
#         return RoombaAct(0, randomAngle)
#     else
#         return RoombaAct(v, th)
#     end

#     return RoombaAct(v, th)
end

In [6]:
# first seed the environment
Random.seed!(2)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
#for (t, step) in enumerate(stepthrough(mdp, p, belief_updater, max_steps=100))
for (t, step) in enumerate(stepthrough(mdp, policy max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, mdp, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

LoadError: syntax: missing comma or ) in argument list

## Evaluation
Here, we demonstate a simple evaluation of the policy's performance for a few random seeds. This is meant to serve only as an example, and we encourage you to develop your own evaluation metrics.
We intialize the robot using five different random seeds, and simulate its performance for 100 time-steps. We then sum the rewards experienced during its interaction with the environment and track this total reward for the five trials. Finally, we report the mean and standard error for the total reward. The standard error is the standard deviation of a sample set divided by the square root of the number of samples, and represents the uncertainty in the estimate of the mean value.

In [7]:
using Statistics

total_rewards = []
println("stepthrough")
for exp = 1:5
    println(string(exp))

    Random.seed!(exp)
    
    p = ToEnd(0)
    #print(step.r for step in stepthrough(m,p,belief_updater, max_steps=100))
    traj_rewards = sum([step.r for step in stepthrough(mdp,policy, max_steps=100)])
    #traj_rewards = sum([step.r for step in stepthrough(mdp,p,belief_updater, max_steps=100)])
    
    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

stepthrough
1
2
3
4
5
Mean Total Reward: 8.300, StdErr Total Reward: 0.126